# Load enviroment and check for folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Clone to Google Drive
!git clone https://github.com/guilherme-webster/NavierStokes-parallel.git /content/gdrive/MyDrive/NavierStokes-parallel

In [ ]:
import subprocess
import os
base_dir = "/content/gdrive/MyDrive/NavierStokes-parallel"
get_ipython().run_line_magic('cd', base_dir)

# Running and comparing with serial

In [ ]:
import numpy as np
import subprocess
import os

# set exercise path
exercise_dir = base_dir

# Function to run shell commands in a specific directory
def run_command(command, work_dir):
    return subprocess.run(command, shell=True, cwd=work_dir, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to compare two output files with numerical tolerance
def compare_outputs_with_tolerance(file1, file2, tolerance=1e-4):
    """
    Compare two output files containing numerical data with a tolerance.
    Returns True if files are equivalent within tolerance, False otherwise.
    """
    try:
        with open(file1, 'r') as f1, open(file2, 'r') as f2:
            lines1 = f1.readlines()
            lines2 = f2.readlines()
        
        # Check if number of lines match
        if len(lines1) != len(lines2):
            return False, f"Different number of lines: {len(lines1)} vs {len(lines2)}"
        
        for i, (line1, line2) in enumerate(zip(lines1, lines2)):
            # Skip empty lines or lines that are identical
            line1, line2 = line1.strip(), line2.strip()
            if line1 == line2:
                continue
            
            # Try to parse as numerical data
            try:
                # Split lines into tokens
                tokens1 = line1.split()
                tokens2 = line2.split()
                
                if len(tokens1) != len(tokens2):
                    return False, f"Line {i+1}: Different number of values: {len(tokens1)} vs {len(tokens2)}"
                
                # Compare each token numerically
                for j, (token1, token2) in enumerate(zip(tokens1, tokens2)):
                    try:
                        val1 = float(token1)
                        val2 = float(token2)
                        
                        # Use relative tolerance for large numbers, absolute for small
                        if max(abs(val1), abs(val2)) > 1.0:
                            if abs(val1 - val2) / max(abs(val1), abs(val2)) > tolerance:
                                return False, f"Line {i+1}, token {j+1}: {val1} vs {val2} (rel diff: {abs(val1-val2)/max(abs(val1),abs(val2)):.2e})"
                        else:
                            if abs(val1 - val2) > tolerance:
                                return False, f"Line {i+1}, token {j+1}: {val1} vs {val2} (abs diff: {abs(val1-val2):.2e})"
                    except ValueError:
                        # Non-numerical tokens must match exactly
                        if token1 != token2:
                            return False, f"Line {i+1}, token {j+1}: Non-numerical tokens differ: '{token1}' vs '{token2}'"
            except:
                # If parsing fails, compare as strings
                if line1 != line2:
                    return False, f"Line {i+1}: String comparison failed: '{line1}' vs '{line2}'"
        
        return True, "Files match within tolerance"
    except Exception as e:
        return False, f"Error reading files: {str(e)}"

# Create and prepare the build directory
cmd_return = run_command('cmake -E remove -f build', exercise_dir)
print(f"Cmake output:\n{cmd_return.stdout}")

if cmd_return.returncode != 0:
    print(f"Cmake failed with error:\n{cmd_return.stderr}")
    assert False

run_command('cmake -E make_directory build', exercise_dir)
cmd_return = run_command('cmake ../ -DCMAKE_BUILD_TYPE=Release -DARCH=sm_75', os.path.join(exercise_dir, 'build'))
print(f"Cmake output:\n{cmd_return.stdout}")

if cmd_return.returncode != 0:
    print(f"Cmake failed with error:\n{cmd_return.stderr}")
    assert False

# Compile the programs
cmd_return = run_command('make', os.path.join(exercise_dir, 'build'))
print(f"Compilation output:\n{cmd_return.stdout}")

if cmd_return.returncode != 0:
    print(f"Compilation failed with error:\n{cmd_return.stderr}")
    assert False

# Execute tests and collect outputs
tolerance = 1e-4  # Tolerance for numerical comparison
for i in range(1, 6):
    test_input = f"tests/{i}.in"

    if not os.path.exists(os.path.join(exercise_dir, test_input)):
        print("Error! Test input not found")
        continue

    parallel_output = f"parallel.{i}.out"
    serial_output = f"serial.{i}.out"
    parallel_time = f"parallel.{i}.time"
    serial_time = f"serial.{i}.time"


    # Workaround for GDrive permissions
    subprocess.run(f"chmod 755 ./parallel", shell=True, cwd=os.path.join(exercise_dir, 'build'))
    subprocess.run(f"chmod 755 ./serial", shell=True, cwd=os.path.join(exercise_dir, 'build'))

    # Run parallel and serial programs
    subprocess.run(f"build/parallel {test_input} > build/{parallel_output} 2> build/{parallel_time}", shell=True, cwd=os.path.join(exercise_dir))
    subprocess.run(f"build/serial {test_input} > build/{serial_output} 2> build/{serial_time}", shell=True, cwd=os.path.join(exercise_dir))

    # Compare outputs with numerical tolerance
    serial_file = os.path.join(exercise_dir, 'build', serial_output)
    parallel_file = os.path.join(exercise_dir, 'build', parallel_output)
    
    files_match, error_msg = compare_outputs_with_tolerance(serial_file, parallel_file, tolerance)

    # Read timings and calculate speedup
    with open(os.path.join(exercise_dir, 'build', serial_time), 'r') as f:
        serial_runtime = float(f.read().strip())

    test_status = "[CORRECT OUTPUT]" if files_match else "[INCORRECT OUTPUT]"

    # Read parallel runtime only if the output is correct
    if files_match:
        with open(os.path.join(exercise_dir, 'build', parallel_time), 'r') as f:
            parallel_runtime = float(f.read().strip())
        speedup = serial_runtime / parallel_runtime if parallel_runtime > 0 else float('inf')
        print(f"Test {i}: {test_status} Serial time = {serial_runtime}s, Parallel time = {parallel_runtime}s, Speedup = {speedup:.4f}x\n")

    else:
        print(f"Test {i}: {test_status} Serial time = {serial_runtime}s, Parallel time = --, Speedup = --")
        print(f"Comparison error: {error_msg}\n\n")